In [1]:
#Imports
import sys
sys.path.append('../../')

from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .config("spark.jar", "lib/sparknlp.jar") \
    .config("spark.driver.memory", "5g")\
    .config("spark.dirver.maxResultSize", "2g")\
    .getOrCreate()

In [3]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

+------+---------+--------------------+
|itemid|sentiment|                text|
+------+---------+--------------------+
|     1|        0|                 ...|
|     2|        0|                 ...|
|     3|        1|              omg...|
|     4|        0|          .. Omga...|
|     5|        0|         i think ...|
|     6|        0|         or i jus...|
|     7|        1|       Juuuuuuuuu...|
|     8|        0|       Sunny Agai...|
|     9|        1|      handed in m...|
|    10|        1|      hmmmm.... i...|
|    11|        0|      I must thin...|
|    12|        1|      thanks to a...|
|    13|        0|      this weeken...|
|    14|        0|     jb isnt show...|
|    15|        0|     ok thats it ...|
|    16|        0|    &lt;-------- ...|
|    17|        0|    awhhe man.......|
|    18|        1|    Feeling stran...|
|    19|        0|    HUGE roll of ...|
|    20|        0|    I just cut my...|
+------+---------+--------------------+
only showing top 20 rows



In [4]:
### Define the dataframe
document_assembler = DocumentAssembler() \
            .setInputCol("text")
        
### Transform input to appropriate schema
#assembled = document_assembler.transform(data)

In [5]:
### Sentence detector
sentence_detector = SentenceDetectorModel() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
#sentence_data = sentence_detector.transform(checked)

In [6]:
### Tokenizer
tokenizer = RegexTokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
#tokenized = tokenizer.transform(assembled)

In [7]:
### Spell Checker
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["token"]) \
            .setOutputCol("spell")

#checked = spell_checker.fit(tokenized).transform(tokenized)

In [8]:
sentiment_detector = ViveknSentimentApproach() \
    .setInputCols(["spell", "sentence"]) \
    .setOutputCol("sentiment") \
    .setPositiveSource("../../../src/test/resources/vivekn/positive") \
    .setNegativeSource("../../../src/test/resources/vivekn/negative") \
    .setPruneCorpus(False)


In [9]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeKeys(True)

In [10]:
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    spell_checker,
    sentiment_detector,
    finisher
])

sentiment_data = pipeline.fit(data).transform(data)
sentiment_data.show()

+------+--------------------+--------------------+
|itemid|                text|  finished_sentiment|
+------+--------------------+--------------------+
|     1|                 ...|sentiment -> posi...|
|     2|                 ...|sentiment -> posi...|
|     3|              omg...|sentiment -> posi...|
|     4|          .. Omga...|sentiment -> posi...|
|     5|         i think ...|sentiment -> posi...|
|     6|         or i jus...|sentiment -> posi...|
|     7|       Juuuuuuuuu...|sentiment -> posi...|
|     8|       Sunny Agai...|sentiment -> posi...|
|     9|      handed in m...|sentiment -> posi...|
|    10|      hmmmm.... i...|sentiment -> posi...|
|    11|      I must thin...|sentiment -> posi...|
|    12|      thanks to a...|sentiment -> posi...|
|    13|      this weeken...|sentiment -> posi...|
|    14|     jb isnt show...|sentiment -> posi...|
|    15|     ok thats it ...|sentiment -> posi...|
|    16|    &lt;-------- ...|sentiment -> posi...|
|    17|    awhhe man.......|se

In [11]:
for r in sentiment_data.take(5):
    print(r)

Row(itemid=1, text='                     is so sad for my APL friend.............', finished_sentiment='sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive')
Row(itemid=2, text='                   I missed the New Moon trailer...', finished_sentiment='sentiment -> positive@sentiment -> positive@sentiment -> positive')
Row(itemid=3, text='              omg its already 7:30 :O', finished_sentiment='sentiment -> positive')
Row(itemid=4, text="          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...", finished_sentiment='sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> positive@sentiment -> negative@sentiment -> positive@sentiment -> 

In [12]:
pipeline.write().overwrite().save("./ps")
pipeline.fit(data).write().overwrite().save("./ms")

In [13]:
from pyspark.ml import Pipeline,PipelineModel

In [14]:
Pipeline.read().load("./ps")
PipelineModel.read().load("./ms")

PipelineModel_4acdba3a7af616d553cc